In [3]:
import torch
import model
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import matplotlib.image as mpimg
import pandas as pd
import torch
from torchvision import transforms, datasets
%matplotlib inline

In [ ]:
img1 = mpimg.imread('data/train/cat.0.png')
img2 = mpimg.imread('data/train/cat.1000.png')
img3 = mpimg.imread('data/train/dog.10000.png')
img4 = mpimg.imread('data/train/dog.1001.png')

In [ ]:

fig = plt.figure()
a = fig.add_subplot(1, 2, 1)
imgplot = plt.imshow(img1)
a.set_title('img1')
a = fig.add_subplot(1, 2, 2)
imgplot = plt.imshow(img2)
a.set_title('img2')


In [ ]:
fig1 = plt.figure()
a = fig1.add_subplot(1, 2, 1)
imgplot = plt.imshow(img3)
a.set_title('img3')
a = fig1.add_subplot(1, 2, 2)
imgplot = plt.imshow(img4)
a.set_title('img4')


In [ ]:
from torch.utils.data import Dataset, DataLoader

class CatandDog(Dataset):
    

    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.labels= pd.read_csv(csv_file)
       
      
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        
        image_name = os.path.join(self.root_dir,
                                self.labels.iloc[idx, 0])
        #print(image_name)

        image = mpimg.imread(image_name)
#         print(image.shape)
        
        #if image has an alpha color channel, get rid of it
        if(image.shape[2] == 4):
            image = image[:,:,0:3]
        
        label= self.labels.iloc[idx, 1]
    
        #self.sample = ['image', 'labels']
        if label==0:
            label_=[1,0]
        else:
            label_=[0,1]
        sample = {'image': image, 'labels': label_}
        

        if self.transform:
            sample = self.transform(sample)

        return sample


In [ ]:
#Constructing the dataset
cat_dog_dataset = CatandDog(csv_file='train.csv',
                                      root_dir='/data/train')

# print some stats about the dataset


train_set, val_set = torch.utils.data.random_split(cat_dog_dataset, [20000, 4999]) # splitting training data into training, test, cross validation

train_loader = DataLoader(val_set, 
                          batch_size=32,
                          shuffle=True, 
                          num_workers=4)
print('Length of dataset: ', len(train_set))

#### Visualizing some images from dataset

In [ ]:
cat_dog_dataset=CatandDog(csv_file='train.csv',
                                      root_dir='data/train')
# rand_i = np.random.randint(0, len(cat_dog))
# sample = cat_dog[rand_i]
#print( sample['image'].shape, sample['labels'].shape)
#sample['image'].shape

for i in range(10):
    
    # define the size of images
    fig = plt.figure(figsize=(20,10))
    
    # randomly select a sample
    rand_i = np.random.randint(0, len(cat_dog_dataset))
    sample = cat_dog_dataset[rand_i]

    # print the shape of the image and keypoints
    print(i, sample['image'].shape, sample['labels'])
    plt.imshow(sample['image'])
    



# Image Preprocessing


<item>\Normalize: to convert a color image to grayscale values with a range of [0,1] and normalize the keypoints to be in a range of about [-1, 1]
<item>Rescale: to rescale an image to a desired size.
<item>RandomCrop: to crop an image randomly.
<item>ToTensor: to convert numpy images to torch images.
   <p> We  wrote them as callable classes instead of simple functions so that parameters of the transform need not be passed everytime it's called. For this, we just need to implement __call__ method and (if we require parameters to be passed in), the __init__ method. We can then use a transform like this:

tx = Transform(params)
transformed_sample = tx(sample)\<p>
    

In [ ]:
import torch
from torchvision import transforms, utils
# tranforms

class Normalize(object):
    """Convert a color image to grayscale and normalize the color range to [0,1]."""        

    def __call__(self, sample):
        image = sample['image']
        label = sample['labels']
        image_copy = np.copy(image)
        
        # convert image to grayscale
#         image_copy = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        
        # scale color range from [0, 255] to [0, 1]
        image_copy=  image_copy/255.0

#         print(image_copy.shape)
        return {'image': image_copy, 'labels': label}


class Rescale(object):
    """Rescale the image in a sample to a given size.

    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image = sample['image']
        label = sample['labels']
        h, w = image.shape[:2]
        if isinstance(self.output_size, int):  # to check if the output_size is of int type
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = cv2.resize(image, (new_w, new_h))
        
       

        return {'image': img, 'labels': label} 


class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image = sample['image']
        label = sample['labels']
        h, w = image.shape[0:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]

        return {'image': image, 'labels': label}


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image = sample['image']
        label = sample['labels']
        # if image has no grayscale color channel, add one
        if(len(image.shape) == 2):
            # add that third color dim
            image = image.reshape(image.shape[0], image.shape[1], 1)
            
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image), 'labels':label}

### Testing operations on some samples

In [ ]:
rescale = Rescale(100)
crop = RandomCrop(50)
composed = transforms.Compose([Rescale(250),
                               RandomCrop(224)])

# apply the transforms to a sample image
test_num = 1000
sample = cat_dog_dataset[test_num]

fig = plt.figure()
for i, tx in enumerate([rescale, crop, composed]):
    transformed_sample = tx(sample)

    ax = plt.subplot(1, 3, i + 1)
    print(transformed_sample['image'].shape, sample['labels'])
    plt.imshow(sample['image'])
#     plt.tight_layout()
    ax.set_title(type(tx).__name__)
    plt.imshow(transformed_sample['image'])#

plt.show()


In [ ]:
# order matters! i.e. rescaling should come before a smaller crop
data_transform = transforms.Compose([Rescale(250),
                                     RandomCrop(224),
                                     Normalize(),
                                     ToTensor()])

# create the transformed dataset
transformed_dataset = CatandDog(csv_file='train.csv',
                                      root_dir='data/train',transform=data_transform)

In [ ]:
# print some stats about the transformed data
print('Number of images: ', len(transformed_dataset))

In [ ]:
for i in range(5):
    sample = transformed_dataset[i]
    print(i, sample['image'].size(), sample['labels'])

In [ ]:
from model import Net

In [ ]:
net=Net()
print(net)

In [ ]:
path= 'model_4_val(1)'
model = Net()
model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
model.eval()

In [2]:
def test(model):
    net.eval()
    for i, (data) in enumerate(train_loader):
        images = data['image']
        # print(labels)
        
        images = images.type(torch.FloatTensor)
        # #transferring into cuda
        images=images.to(device)
        # print(images.shape)


        # Predict classes using images from the test set
        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        print(prediction)


In [ ]:
path=''

In [ ]:
torch.load_state_dict(path)
